In [1]:
import numpy as np
import pandas as pd
import csv as csv

In [2]:
from scipy import spatial, optimize
import scipy

In [104]:
from collections import defaultdict

In [38]:
cd /Users/tianhaolu/Documents/Playground/homedepot_tfidf

/Users/tianhaolu/Documents/Playground/homedepot_tfidf


In [4]:
data=pd.read_csv("homedepot_ads.csv")

In [5]:
data.loc[1:97][['title','call_to_action']]

,title,call_to_action
1,Easy-to-Install Doors,Available at The Home Depot
2,Prevent Weather Damage,Click to Learn How
3,Hurricane Season Is Near,Be Prepared w/ The Home Depot
4,Be Ready for Bad Weather,Get Prepared w/ The Home Depot
5,Bad Weather In Your Area?,Click to Learn How
6,Want Your Doors to Last?,Find Out How
7,Storm-Ready in 45 Minutes,Get Prepared at The Home Depot
8,Transform Your Entryway,Click to Browse
9,Protect Your Home,Find Out How
10,New Doors in No Time!,Click to Browse


In [6]:
data.shape

(96, 12)

In [28]:
data=data.drop(['Desktop'],1)

In [7]:
data['display_name'].unique()

array(['desktop 300x250', 'desktop 728x90', 'mobile 300x250',
       'mobile 320x50'], dtype=object)

In [8]:
X=data[data['display_name'].isin(['desktop 300x250','desktop 728x90'])]

In [11]:
Y=Y[Y['display_name'].isin(['desktop 300x250'])]

In [78]:
X.loc[16]['description'].split()[5]

'Andersen\xa8'

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
vect = TfidfVectorizer(min_df=1)
new_docs=["I'd like an apple apple apple apple apple apple apple",
                             "An apple a day keeps the doctor away",
                        "Never compare an apple to an orange",
                            "I prefer scikit-learn to Orange",
                           "datarama","Datarama"]
tfidf = vect.fit_transform(new_docs)

In [26]:
(tfidf * tfidf.T).A

array([[ 1.        ,  0.31438232,  0.36030996,  0.        ,  0.        ,
         0.        ],
       [ 0.31438232,  1.        ,  0.24583709,  0.        ,  0.        ,
         0.        ],
       [ 0.36030996,  0.24583709,  1.        ,  0.26926468,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.26926468,  1.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         1.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         1.        ]])

In [22]:
new_term_freq_matrix = vect.transform(new_docs)

In [25]:
vect.vocabulary_

{u'an': 0,
 u'apple': 1,
 u'away': 2,
 u'compare': 3,
 u'datarama': 4,
 u'day': 5,
 u'doctor': 6,
 u'keeps': 7,
 u'learn': 8,
 u'like': 9,
 u'never': 10,
 u'orange': 11,
 u'prefer': 12,
 u'scikit': 13,
 u'the': 14,
 u'to': 15}

In [27]:
print new_term_freq_matrix.todense()

[[ 0.13856     0.96991998  0.          0.          0.          0.          0.
   0.          0.          0.20014085  0.          0.          0.          0.
   0.          0.        ]
 [ 0.2836157   0.2836157   0.40966432  0.          0.          0.40966432
   0.40966432  0.40966432  0.          0.          0.          0.          0.
   0.          0.40966432  0.        ]
 [ 0.57786433  0.28893217  0.          0.41734361  0.          0.          0.
   0.          0.          0.          0.41734361  0.34222773  0.          0.
   0.          0.34222773]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.47974754  0.          0.          0.39339985  0.47974754
   0.47974754  0.          0.39339985]
 [ 0.          0.          0.          0.          1.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          1.          0.    

In [10]:
Y=X.copy()

In [55]:
Y['label']=range(25)


/Users/tianhaolu/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [12]:
Y = Y.reset_index(drop=True)

In [13]:
Y

,id,dimensions,display_name,title,description,call_to_action,external_placement_id,external_placement_name,ct_url,beacon_urls,script_urls,enabled
0,560046b60b9f73e808486989,300x250,desktop 300x250,Protect Your Home,Pre-assembled Storm Doors Up & Ready in Less T...,Find Out How,122516991,YIELDBOT�Run of Network �300 x 250 (NA)�SS�Tex...,https://ad.doubleclick.net/ddm/clk/295631116;1...,https://ad.doubleclick.net/ddm/ad/N7310.122508...,NaN,True
1,560046b60b9f73e808486986,300x250,desktop 300x250,Easy-to-Install Doors,It Doesn't Have to Be a Hassle w/ the Help of ...,Available at The Home Depot,122516991,YIELDBOT�Run of Network �300 x 250 (NA)�SS�Tex...,https://ad.doubleclick.net/ddm/clk/295631116;1...,https://ad.doubleclick.net/ddm/ad/N7310.122508...,NaN,True
2,560046b60b9f73e80848698f,300x250,desktop 300x250,Prevent Weather Damage,Protect Your Home from the Elements w/ Easy-to...,Click to Learn How,122516991,YIELDBOT�Run of Network �300 x 250 (NA)�SS�Tex...,https://ad.doubleclick.net/ddm/clk/295631116;1...,https://ad.doubleclick.net/ddm/ad/N7310.122508...,NaN,True
3,560046b60b9f73e808486990,300x250,desktop 300x250,Hurricane Season Is Near,Pre-assembled Storm Doors Up & Ready in No Time,Be Prepared w/ The Home Depot,122516991,YIELDBOT�Run of Network �300 x 250 (NA)�SS�Tex...,https://ad.doubleclick.net/ddm/clk/295631116;1...,https://ad.doubleclick.net/ddm/ad/N7310.122508...,NaN,True
4,560046b60b9f73e80848698c,300x250,desktop 300x250,Be Ready for Bad Weather,Pre-assembled Storm Doors Up & Ready in No Time,Get Prepared w/ The Home Depot,122516991,YIELDBOT�Run of Network �300 x 250 (NA)�SS�Tex...,https://ad.doubleclick.net/ddm/clk/295631116;1...,https://ad.doubleclick.net/ddm/ad/N7310.122508...,NaN,True
5,560046b60b9f73e808486993,300x250,desktop 300x250,Bad Weather In Your Area?,Protect Your Home from the Elements w/ Easy-to...,Click to Learn How,122516991,YIELDBOT�Run of Network �300 x 250 (NA)�SS�Tex...,https://ad.doubleclick.net/ddm/clk/295631116;1...,https://ad.doubleclick.net/ddm/ad/N7310.122508...,NaN,True
6,560046b60b9f73e808486996,300x250,desktop 300x250,Want Your Doors to Last?,Protect Your Exterior Doors from the Elements ...,Find Out How,122516991,YIELDBOT�Run of Network �300 x 250 (NA)�SS�Tex...,https://ad.doubleclick.net/ddm/clk/295631116;1...,https://ad.doubleclick.net/ddm/ad/N7310.122508...,NaN,True
7,560046b60b9f73e808486999,300x250,desktop 300x250,Transform Your Entryway,Complement Your Beautifully Crafted Front Door...,Click to Browse,122516991,YIELDBOT�Run of Network �300 x 250 (NA)�SS�Tex...,https://ad.doubleclick.net/ddm/clk/295631116;1...,https://ad.doubleclick.net/ddm/ad/N7310.122508...,NaN,True
8,560046b60b9f73e808486985,300x250,desktop 300x250,Storm-Ready in 45 Minutes,Pre-assembled Doors Up & Ready in No Time,Get Prepared at The Home Depot,122516991,YIELDBOT�Run of Network �300 x 250 (NA)�SS�Tex...,https://ad.doubleclick.net/ddm/clk/295631116;1...,https://ad.doubleclick.net/ddm/ad/N7310.122508...,NaN,True
9,560046b60b9f73e808486988,300x250,desktop 300x250,New Doors in No Time!,Get Easy & Quick Storm Door Installation w/ Th...,Click to Browse,122516991,YIELDBOT�Run of Network �300 x 250 (NA)�SS�Tex...,https://ad.doubleclick.net/ddm/clk/295631116;1...,https://ad.doubleclick.net/ddm/ad/N7310.122508...,NaN,True


In [19]:
vect = TfidfVectorizer(min_df=1)

In [27]:
Y['title']=map(lambda x: x.replace("!",""),list(Y['title']))
Y['title']=map(lambda x: x.replace("?",""),list(Y['title']))
Y['call_to_action']=map(lambda x: x.replace("!",""),list(Y['call_to_action']))
Y['call_to_action']=map(lambda x: x.replace("?",""),list(Y['call_to_action']))

In [15]:
Y['space']=' '

In [28]:
Y['string'] = Y[['title','space','call_to_action']].apply(lambda x: ''.join(x), axis=1)

In [29]:
tfidf = vect.fit_transform(list(Y['string']))

In [30]:
dist=pd.DataFrame((tfidf * tfidf.T).A)

In [31]:
dist[1][0]

0.064724012654217219

In [32]:
dist

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,1.000000,0.064724,0.109454,0.053616,0.056684,0.212145,0.588474,0.128955,0.052715,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.162540,0.160936,0.054509,0.067725,0.301160,0.000000
1,0.064724,1.000000,0.079689,0.189690,0.200544,0.072250,0.122210,0.082518,0.294294,0.124069,...,0.058839,0.058839,0.291779,0.070849,0.050760,0.128400,0.192848,0.062068,0.128091,0.310877
2,0.109454,0.079689,1.000000,0.000000,0.135520,0.476612,0.163908,0.164138,0.000000,0.147339,...,0.000000,0.000000,0.000000,0.000000,0.068194,0.344217,0.000000,0.119423,0.356782,0.000000
3,0.053616,0.189690,0.000000,1.000000,0.398770,0.000000,0.000000,0.000000,0.243788,0.000000,...,0.000000,0.000000,0.000000,0.058691,0.042049,0.045246,0.252085,0.051416,0.000000,0.000000
4,0.056684,0.200544,0.135520,0.398770,1.000000,0.271588,0.000000,0.000000,0.455938,0.000000,...,0.085321,0.085321,0.000000,0.131393,0.143643,0.154565,0.332555,0.054358,0.000000,0.000000
5,0.212145,0.072250,0.476612,0.000000,0.271588,1.000000,0.242833,0.255382,0.114265,0.263282,...,0.000000,0.000000,0.000000,0.000000,0.150377,0.312083,0.000000,0.108275,0.323476,0.000000
6,0.588474,0.122210,0.163908,0.000000,0.000000,0.242833,1.000000,0.182759,0.000000,0.112978,...,0.053579,0.053579,0.045076,0.000000,0.089422,0.151754,0.000000,0.000000,0.204297,0.048027
7,0.128955,0.082518,0.164138,0.000000,0.000000,0.255382,0.182759,1.000000,0.000000,0.331863,...,0.000000,0.000000,0.000000,0.000000,0.171748,0.151967,0.000000,0.000000,0.157515,0.000000
8,0.052715,0.294294,0.000000,0.243788,0.455938,0.114265,0.000000,0.000000,1.000000,0.117147,...,0.168717,0.168717,0.000000,0.194828,0.041342,0.044485,0.309268,0.050551,0.000000,0.000000
9,0.000000,0.124069,0.147339,0.000000,0.000000,0.263282,0.112978,0.331863,0.117147,1.000000,...,0.054395,0.054395,0.045762,0.000000,0.063387,0.249075,0.000000,0.000000,0.189111,0.168075


In [24]:
Y.loc[1]['string']

'Easy-to-Install Doors Available at The Home Depot'

In [84]:
edge=[]
for i in range(25):
    for j in range(25):
        if i!=j:
            if dist[i].loc[j]>0:
                edge.append([i,j,Y.loc[i]['string'],Y.loc[j]['string'],dist[i].loc[j]])

In [85]:
with open("homedepot_tfidf/edge.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(edge)

In [49]:
len(Y['string'].unique())

23

In [11]:
Y['space']=' '

In [33]:
Y['string']

0                        Protect Your Home Find Out How
1     Easy-to-Install Doors Available at The Home Depot
2             Prevent Weather Damage Click to Learn How
3     Hurricane Season Is Near Be Prepared w/ The Ho...
4     Be Ready for Bad Weather Get Prepared w/ The H...
5           Bad Weather In Your Area Click to Learn How
6                  Want Your Doors to Last Find Out How
7               Transform Your Entryway Click to Browse
8     Storm-Ready in 45 Minutes Get Prepared at The ...
9                  New Doors in No Time Click to Browse
10             Why Buy a Storm Door Click for More Info
11     Replacing Doors Is Costly Save at The Home Depot
12             Before It's Too Late Get Storm Doors Now
13     Stylish & Practical Doors Save at The Home Depot
14         Need a New Front Door Try Easy Install Doors
15         Exterior Door Protection Get Storm Doors Now
16         Exterior Door Protection Get Storm Doors Now
17      Hassle-Free Installation Try Easy Instal

In [86]:
Y.to_csv('homedepot_tfidf/node.csv')

In [34]:
Y

,id,dimensions,display_name,title,description,call_to_action,external_placement_id,external_placement_name,ct_url,beacon_urls,script_urls,enabled,space,string
0,560046b60b9f73e808486989,300x250,desktop 300x250,Protect Your Home,Pre-assembled Storm Doors Up & Ready in Less T...,Find Out How,122516991,YIELDBOT�Run of Network �300 x 250 (NA)�SS�Tex...,https://ad.doubleclick.net/ddm/clk/295631116;1...,https://ad.doubleclick.net/ddm/ad/N7310.122508...,NaN,True,,Protect Your Home Find Out How
1,560046b60b9f73e808486986,300x250,desktop 300x250,Easy-to-Install Doors,It Doesn't Have to Be a Hassle w/ the Help of ...,Available at The Home Depot,122516991,YIELDBOT�Run of Network �300 x 250 (NA)�SS�Tex...,https://ad.doubleclick.net/ddm/clk/295631116;1...,https://ad.doubleclick.net/ddm/ad/N7310.122508...,NaN,True,,Easy-to-Install Doors Available at The Home Depot
2,560046b60b9f73e80848698f,300x250,desktop 300x250,Prevent Weather Damage,Protect Your Home from the Elements w/ Easy-to...,Click to Learn How,122516991,YIELDBOT�Run of Network �300 x 250 (NA)�SS�Tex...,https://ad.doubleclick.net/ddm/clk/295631116;1...,https://ad.doubleclick.net/ddm/ad/N7310.122508...,NaN,True,,Prevent Weather Damage Click to Learn How
3,560046b60b9f73e808486990,300x250,desktop 300x250,Hurricane Season Is Near,Pre-assembled Storm Doors Up & Ready in No Time,Be Prepared w/ The Home Depot,122516991,YIELDBOT�Run of Network �300 x 250 (NA)�SS�Tex...,https://ad.doubleclick.net/ddm/clk/295631116;1...,https://ad.doubleclick.net/ddm/ad/N7310.122508...,NaN,True,,Hurricane Season Is Near Be Prepared w/ The Ho...
4,560046b60b9f73e80848698c,300x250,desktop 300x250,Be Ready for Bad Weather,Pre-assembled Storm Doors Up & Ready in No Time,Get Prepared w/ The Home Depot,122516991,YIELDBOT�Run of Network �300 x 250 (NA)�SS�Tex...,https://ad.doubleclick.net/ddm/clk/295631116;1...,https://ad.doubleclick.net/ddm/ad/N7310.122508...,NaN,True,,Be Ready for Bad Weather Get Prepared w/ The H...
5,560046b60b9f73e808486993,300x250,desktop 300x250,Bad Weather In Your Area,Protect Your Home from the Elements w/ Easy-to...,Click to Learn How,122516991,YIELDBOT�Run of Network �300 x 250 (NA)�SS�Tex...,https://ad.doubleclick.net/ddm/clk/295631116;1...,https://ad.doubleclick.net/ddm/ad/N7310.122508...,NaN,True,,Bad Weather In Your Area Click to Learn How
6,560046b60b9f73e808486996,300x250,desktop 300x250,Want Your Doors to Last,Protect Your Exterior Doors from the Elements ...,Find Out How,122516991,YIELDBOT�Run of Network �300 x 250 (NA)�SS�Tex...,https://ad.doubleclick.net/ddm/clk/295631116;1...,https://ad.doubleclick.net/ddm/ad/N7310.122508...,NaN,True,,Want Your Doors to Last Find Out How
7,560046b60b9f73e808486999,300x250,desktop 300x250,Transform Your Entryway,Complement Your Beautifully Crafted Front Door...,Click to Browse,122516991,YIELDBOT�Run of Network �300 x 250 (NA)�SS�Tex...,https://ad.doubleclick.net/ddm/clk/295631116;1...,https://ad.doubleclick.net/ddm/ad/N7310.122508...,NaN,True,,Transform Your Entryway Click to Browse
8,560046b60b9f73e808486985,300x250,desktop 300x250,Storm-Ready in 45 Minutes,Pre-assembled Doors Up & Ready in No Time,Get Prepared at The Home Depot,122516991,YIELDBOT�Run of Network �300 x 250 (NA)�SS�Tex...,https://ad.doubleclick.net/ddm/clk/295631116;1...,https://ad.doubleclick.net/ddm/ad/N7310.122508...,NaN,True,,Storm-Ready in 45 Minutes Get Prepared at The ...
9,560046b60b9f73e808486988,300x250,desktop 300x250,New Doors in No Time,Get Easy & Quick Storm Door Installation w/ Th...,Click to Browse,122516991,YIELDBOT�Run of Network �300 x 250 (NA)�SS�Tex...,https://ad.doubleclick.net/ddm/clk/295631116;1...,https://ad.doubleclick.net/ddm/ad/N7310.122508...,NaN,True,,New Doors in No Time Click to Browse


In [39]:
MG=pd.read_csv("homedepot_MG.csv")

In [40]:
MG

,id,display_name,type,topic,enabled,keywords,psns,verticals
0,560042790b9f73e808486977,V home garden,standard,NaN,True,"is,our,from,ideas,fall,tips,make,up,photos,diy...",NaN,"537eba499c5f10b715000005,53cfe1b6931e9f8618000..."
1,560042790b9f73e80848697a,gym,standard,NaN,True,"workout,work out,exercises,calories,weights,we...",NaN,"537eba499c5f10b715000005,53cfe1b6931e9f8618000..."
2,560045b70b9f73e808486981,mobile home garden vertical,standard,NaN,True,"is,our,from,ideas,fall,tips,make,up,photos,diy...",NaN,"537eba499c5f10b715000005,53cfe1b6931e9f8618000..."
3,560045b70b9f73e808486984,mobile tool,standard,NaN,True,"tool,tools,battery,driver,impact driver,kit,to...",NaN,"537eba499c5f10b715000005,53cfe1b6931e9f8618000..."
4,56004ddb0b9f73e8084869e1,weather,standard,NaN,True,"weather,bad weather,hurricane,hurricanes",NaN,"537eba499c5f10b715000005,53cfe1b6931e9f8618000..."
5,5602b720d1ddf44c37b4bdd2,untarget men's life,untarget,NaN,True,NaN,NaN,53cfe1b6931e9f8618000001
6,560407aa767e372b056e0e0f,V family,standard,NaN,True,"babycenter,pregnancy,my,kids,baby,about,birth,...",NaN,537eb7d549e709c515000003
7,56046824691b340709b74d09,V auto,standard,NaN,True,"cars,car,how,new,news,culture,auto,under,hood,...",NaN,53ced7179950584e2f000001
8,5605bd0bb7be7a9e460c24cd,you,standard,NaN,True,"you,your,yours",NaN,537eba035f62c9bc15000002
9,5605908cfc5dd7ec2a9a0a54,untarget sports,untarget,NaN,True,NaN,NaN,54ae925586d6228f1f58c49f


In [77]:
mg=MG[pd.notnull(MG['keywords'])]

In [79]:
len(mg)

39

In [84]:
mg=mg[map(lambda x:'mobile' not in x, mg['display_name'])]

In [86]:
mg=mg[map(lambda x:'response' not in x, mg['display_name'])]

In [88]:
mg=mg[map(lambda x:'responsive' not in x, mg['display_name'])]

In [89]:
mg

,id,display_name,type,topic,enabled,keywords,psns,verticals
0,560042790b9f73e808486977,V home garden,standard,NaN,True,"is,our,from,ideas,fall,tips,make,up,photos,diy...",NaN,"537eba499c5f10b715000005,53cfe1b6931e9f8618000..."
1,560042790b9f73e80848697a,gym,standard,NaN,True,"workout,work out,exercises,calories,weights,we...",NaN,"537eba499c5f10b715000005,53cfe1b6931e9f8618000..."
4,56004ddb0b9f73e8084869e1,weather,standard,NaN,True,"weather,bad weather,hurricane,hurricanes",NaN,"537eba499c5f10b715000005,53cfe1b6931e9f8618000..."
6,560407aa767e372b056e0e0f,V family,standard,NaN,True,"babycenter,pregnancy,my,kids,baby,about,birth,...",NaN,537eb7d549e709c515000003
7,56046824691b340709b74d09,V auto,standard,NaN,True,"cars,car,how,new,news,culture,auto,under,hood,...",NaN,53ced7179950584e2f000001
8,5605bd0bb7be7a9e460c24cd,you,standard,NaN,True,"you,your,yours",NaN,537eba035f62c9bc15000002
14,560042790b9f73e808486979,furniture,standard,NaN,True,"door,door frame,wall,lobby,window,window frame...",NaN,"537eba499c5f10b715000005,53cfe1b6931e9f8618000..."
16,560042790b9f73e80848697c,tool,standard,NaN,True,"tool,tools,battery,driver,impact driver,kit,to...",NaN,"537eba499c5f10b715000005,53cfe1b6931e9f8618000..."
21,56040284767e372b056e0e05,disconnectme,standard,NaN,True,"site:facebook.com,monitor,site:linkedin.com,fo...",ea12,NaN
22,5602b65cd1ddf44c37b4bdb8,root veggie,standard,NaN,True,"beet,rubbatage,onion,carrot,yam,yams,plantain,...",NaN,"537eba035f62c9bc15000002,537eba1649e709c515000004"


In [157]:
mg.shape

(22, 9)

In [158]:
mg['idx']=range(mg.shape[0])

/Users/tianhaolu/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [161]:
mg = mg.reset_index(drop=True)

In [128]:
mg['KW']=map(lambda x: x.split(","),mg['keywords'])

/Users/tianhaolu/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [162]:
# Now we have 25 creatives & 22 MGs
mg

,id,display_name,type,topic,enabled,keywords,psns,verticals,KW,idx
0,560042790b9f73e808486977,V home garden,standard,NaN,True,"is,our,from,ideas,fall,tips,make,up,photos,diy...",NaN,"537eba499c5f10b715000005,53cfe1b6931e9f8618000...","[is, our, from, ideas, fall, tips, make, up, p...",0
1,560042790b9f73e80848697a,gym,standard,NaN,True,"workout,work out,exercises,calories,weights,we...",NaN,"537eba499c5f10b715000005,53cfe1b6931e9f8618000...","[workout, work out, exercises, calories, weigh...",1
2,56004ddb0b9f73e8084869e1,weather,standard,NaN,True,"weather,bad weather,hurricane,hurricanes",NaN,"537eba499c5f10b715000005,53cfe1b6931e9f8618000...","[weather, bad weather, hurricane, hurricanes]",2
3,560407aa767e372b056e0e0f,V family,standard,NaN,True,"babycenter,pregnancy,my,kids,baby,about,birth,...",NaN,537eb7d549e709c515000003,"[babycenter, pregnancy, my, kids, baby, about,...",3
4,56046824691b340709b74d09,V auto,standard,NaN,True,"cars,car,how,new,news,culture,auto,under,hood,...",NaN,53ced7179950584e2f000001,"[cars, car, how, new, news, culture, auto, und...",4
5,5605bd0bb7be7a9e460c24cd,you,standard,NaN,True,"you,your,yours",NaN,537eba035f62c9bc15000002,"[you, your, yours]",5
6,560042790b9f73e808486979,furniture,standard,NaN,True,"door,door frame,wall,lobby,window,window frame...",NaN,"537eba499c5f10b715000005,53cfe1b6931e9f8618000...","[door, door frame, wall, lobby, window, window...",6
7,560042790b9f73e80848697c,tool,standard,NaN,True,"tool,tools,battery,driver,impact driver,kit,to...",NaN,"537eba499c5f10b715000005,53cfe1b6931e9f8618000...","[tool, tools, battery, driver, impact driver, ...",7
8,56040284767e372b056e0e05,disconnectme,standard,NaN,True,"site:facebook.com,monitor,site:linkedin.com,fo...",ea12,NaN,"[site:facebook.com, monitor, site:linkedin.com...",8
9,5602b65cd1ddf44c37b4bdb8,root veggie,standard,NaN,True,"beet,rubbatage,onion,carrot,yam,yams,plantain,...",NaN,"537eba035f62c9bc15000002,537eba1649e709c515000004","[beet, rubbatage, onion, carrot, yam, yams, pl...",9


In [106]:
c=Y.copy()

In [107]:
len(c)

25

In [114]:
str(c[c['id']==c['id'].loc[1]]['string']).split(" ")

['1',
 '',
 '',
 '',
 'Easy-to-Install',
 'Doors',
 'Available',
 'at',
 'The',
 'Home',
 'Depot\nName:',
 'string,',
 'dtype:',
 'object']

In [115]:
ad=str(c[c['id']==c['id'].loc[1]]['string']).split(" ")

In [168]:
ad=map(lambda x:x.lower(),ad)

In [169]:
ad

['1',
 '',
 '',
 '',
 'easy-to-install',
 'doors',
 'available',
 'at',
 'the',
 'home',
 'depot\nname:',
 'string,',
 'dtype:',
 'object']

In [173]:
set(ad)

{'',
 '1',
 'at',
 'available',
 'depot\nname:',
 'doors',
 'dtype:',
 'easy-to-install',
 'home',
 'object',
 'string,',
 'the'}

In [164]:
mg[mg['id']==mg['id'].loc[16]]['keywords']

16    bedroom,kitchen,bathroom,toilet,tile,dining ro...
Name: keywords, dtype: object

In [149]:
mg['KW'].loc[0]

1

In [175]:
kw=mg['keywords'].loc[21].split(",")
sorted(kw)

['about',
 'allrecipes',
 'apple',
 'apple crisp oat',
 'apptit',
 'arugula',
 'bacon',
 'bake',
 'baked',
 'baking',
 'banana',
 'beef',
 'best',
 'best cheeseburger',
 'best new',
 'best new paris',
 'bistonomy',
 'blog',
 'bon',
 'bon apptit',
 'bread',
 'bread pizzas',
 'bread pizzas ricotta',
 'butter',
 'cacciatore',
 'cacciatore harissa',
 'cake',
 'can',
 'casserole',
 'cheese',
 'cheeseburger',
 'chef',
 'chicken',
 'chicken cacciatore',
 'chicken cacciatore harissa',
 'chili',
 'chip',
 'chocolate',
 'chops',
 'chorizo',
 'chowhound',
 'confidence',
 'connect',
 'cooker',
 'cookie',
 'cookies',
 'cooking',
 'cool',
 'cool weather cooking',
 'cream',
 'creamy',
 'crisp oat',
 'delicious',
 'dinner',
 'dinner love',
 'dinner love story',
 'dinner table',
 'dishes',
 'easy',
 'entertaining',
 'epicurious',
 'epicurious.com',
 'fastest freshest weeknight',
 'find',
 'food',
 'free',
 'freezer',
 'fresh',
 'freshest',
 'fried',
 'from',
 'fruit',
 'garlic',
 'granola',
 'grilled',

In [124]:
set(kw)

TypeError: unhashable type: 'list'

In [176]:
len(set(ad).intersection(set(kw)))

1

In [207]:
plurals = ['kill','kills','killed','fly','flies','door','doors']

In [180]:
from nltk.stem import *
from nltk.stem.porter import *

In [197]:
from stemming.porter2 import stem
stemmer = PorterStemmer()

In [196]:
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

In [182]:
singles = [stemmer.stem(plural) for plural in plurals]

In [208]:
dick=defaultdict(int)
for i in plurals:
    dick[stem(i)]+=1

In [204]:
lmtzr.lemmatize('flew')

'flew'

In [209]:
dick

defaultdict(<type 'int'>, {'fli': 2, 'door': 2, 'kill': 3})

In [183]:
for i in singles:
    print i

caress
fli
die
mule
deni
die
agre
own
humbl
size
meet
state
siez
item
sensat
tradit
refer
colon
plot


In [276]:
cid_mg_dist=defaultdict(lambda: defaultdict(float))

In [277]:
for i in range(c.shape[0]):
    for j in range(mg.shape[0]):
        ad=str(c['string'].loc[i]).split(" ")
        ad=map(lambda x:x.lower(),ad)
        kw=mg['keywords'].loc[j].split(",")
        kw=map(lambda x:x.lower(),kw)
        ad_dict=defaultdict(int)
        kw_dict=defaultdict(int)
        for w in ad:
            ad_dict[stem(w)]+=1
        for w in kw:
            kw_dict[stem(w)]+=1
        relevance=0
        for w in ad_dict.keys():
            if w in kw_dict.keys():
                plus=(float(ad_dict[w])+float(kw_dict[w]))/2
                relevance+=plus
        if relevance==0.0:
            relevance=float(0.1)
        cid_mg_dist[c['id'].loc[i]][mg['id'].loc[j]]=relevance
        
        
        

In [342]:
cid_mg_dist[c['id'].loc[4]].values()

[1.0,
 0.1,
 1.0,
 3.0,
 0.1,
 0.1,
 3.0,
 0.1,
 0.1,
 0.1,
 0.1,
 1.0,
 0.1,
 1.0,
 1.0,
 2.0,
 0.1,
 0.1,
 0.1,
 1.5,
 0.1,
 0.1]

In [343]:
for x in range(25):
    print cid_mg_dist[c['id'].loc[x]].values()

[0.1, 3.0, 2.0, 2.0, 2.0, 0.1, 3.0, 0.1, 0.1, 0.1, 1.5, 3.0, 0.1, 2.0, 2.0, 3.0, 0.1, 0.1, 0.1, 2.0, 0.1, 0.1]
[0.1, 0.1, 0.1, 3.0, 0.1, 0.1, 1.0, 1.0, 0.1, 0.1, 0.1, 1.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[1.0, 1.0, 1.0, 2.0, 1.0, 0.1, 2.0, 0.1, 0.1, 0.1, 0.1, 1.0, 0.1, 1.0, 2.0, 1.0, 0.1, 0.1, 0.1, 2.0, 0.1, 0.1]
[1.5, 1.0, 0.1, 3.0, 1.0, 0.1, 3.0, 0.1, 0.1, 0.1, 0.1, 2.0, 0.1, 0.1, 2.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[1.0, 0.1, 1.0, 3.0, 0.1, 0.1, 3.0, 0.1, 0.1, 0.1, 0.1, 1.0, 0.1, 1.0, 1.0, 2.0, 0.1, 0.1, 0.1, 1.5, 0.1, 0.1]
[1.0, 2.0, 1.0, 3.0, 2.0, 0.1, 2.0, 0.1, 0.1, 0.1, 1.5, 1.0, 0.1, 2.0, 2.0, 1.0, 0.1, 0.1, 0.1, 2.0, 0.1, 0.1]
[0.1, 3.0, 2.0, 2.0, 2.0, 0.1, 2.0, 1.0, 0.1, 0.1, 1.5, 2.0, 0.1, 2.0, 2.0, 3.0, 0.1, 0.1, 0.1, 2.0, 0.1, 0.1]
[0.1, 1.0, 0.1, 1.0, 1.0, 0.1, 1.0, 0.1, 0.1, 0.1, 1.5, 0.1, 0.1, 1.0, 1.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 1.0, 4.0, 0.1, 0.1, 2.0, 0.1, 0.1, 0.1, 0.1, 1.0, 0.1, 1.0, 1.0, 2.0, 0.1, 0.1, 0.1, 1.5, 0.1, 0.1]
[

In [259]:
#now we have cid_mg_dist,  a nested dict (25x22) #

In [367]:
all=0
for i in range(25):
    all+=sum(dist[i])
    
print all-25,(all-25)/(625-25), 
##average dist between ads=0.101, excluding diagonal elements=1

60.905286515 0.101508810858 300


In [363]:
c_mg_all=0
for x in range(25):
    c_mg_all+=sum(cid_mg_dist.values()[x].values())
print c_mg_all, c_mg_all/float(22*25)

387.8 0.705090909091


In [296]:
print (float(22*8)/25)*(float(22*8)/25)*22


1090.3552

In [463]:
lambda1=1
lambda2=0.1
##scale of lambda1 for c_mg relevance; lambda2 for inner product of assignment vector

In [607]:
ad_assign=defaultdict(list)
for x in c['id']:
    ad_assign[x]=[]

In [606]:
ad_count=defaultdict(int)
for x in c['id']:
    ad_count[x]=0
len(ad_count)

25

In [608]:
mg_count=defaultdict(int)
for y in mg['id']:
    mg_count[y]=0
len(mg_count)

22

In [609]:
mg_assign=defaultdict(list)
for y in mg['id']:
    mg_assign[y]=[]
mg_assign

defaultdict(<type 'list'>, {'56004ddb0b9f73e8084869e1': [], '56059082fc5dd7ec2a9a0a53': [], '563226a15520af0577b30ebb': [], '56040284767e372b056e0e05': [], '56046859691b340709b74d0b': [], '5602b501d1ddf44c37b4bdb7': [], '560042790b9f73e808486977': [], '560042790b9f73e808486979': [], '560042790b9f73e808486978': [], '5605bd0bb7be7a9e460c24cf': [], '5605bd0bb7be7a9e460c24cd': [], '5605bd0bb7be7a9e460c24ce': [], '5602b65cd1ddf44c37b4bdb8': [], '5604a4c7691b340709b74d35': [], '561d44b38c943b5f1225088f': [], '560407aa767e372b056e0e0f': [], '560042790b9f73e80848697c': [], '560042790b9f73e80848697b': [], '560042790b9f73e80848697a': [], '56046824691b340709b74d09': [], '5600428e0b9f73e80848697d': [], '5604080f767e372b056e0e10': []})

In [349]:
#def dxj_avg(cid,mid,dist,c,mg,mg_assign):
#    idx=c[c['id']==cid].index.tolist()[0]
#    total=0
#    for j in mg_assign[mid]:
#        idj=c[c['id']==j].index.tolist()[0]
#        total+=dist[idx][idj]
#    count=len(mg_assign[mid])
#    if count>0:
#        return 1-float(total)/count
#    else:
#        return 0

In [462]:
def dxj_avg(cid,mid,dist,c,mg,mg_assign): 
    n=len(mg_assign[mid])
    if n==0:
        return 0
    else:
        cids_0=mg_assign[mid]
        indices_0=c.loc[c['id'].isin(cids_0)].index.tolist()
        cids_0.append(cid)
        indices_1=c.loc[c['id'].isin(cids_0)].index.tolist()
        new=n+1-dist.loc[indices_1,indices_1].sum().sum()/(n+1)
        old=n-dist.loc[indices_0,indices_0].sum().sum()/n
        return new-old

In [337]:
def c_mg_score(cid,mid,cid_mg_dist):
    return cid_mg_dist[cid][mid]

In [362]:
def x_x(cid,mid,ad_count):
    return ad_count[cid]*2+1

In [361]:
c_mg_score(c['id'].loc[3],mg['id'].loc[0],cid_mg_dist)

3.0

In [456]:
mg_assign['56004ddb0b9f73e8084869e1'].append(c['id'].loc[1])
mg_assign['56004ddb0b9f73e8084869e1'].append(c['id'].loc[3])
mg_assign['56004ddb0b9f73e8084869e1'].append(c['id'].loc[7])
mg_assign['56004ddb0b9f73e8084869e1']
dxj_avg(c['id'][5],'56004ddb0b9f73e8084869e1',dist,c,mg,mg_assign)
#test#

0.8815517119466321

In [443]:
c['id'][5]

'560046b60b9f73e808486993'

In [450]:
cids_0=mg_assign['56004ddb0b9f73e8084869e1']
indices_0=c.loc[c['id'].isin(cids_0)].index.tolist()
cids_0.append(c['id'][5])
print cids_0

['560046b60b9f73e808486986', '560046b60b9f73e808486990', '560046b60b9f73e808486999', '560046b60b9f73e808486993']


In [452]:
c.loc[c['id'].isin(cids_0)].index.tolist()

[1, 3, 5, 7]

In [312]:
len(ad_assign)

25

In [604]:
def flow_assign(c,mg,dist,cid_mg_dist,k,mg_assign,mg_count,ad_assign,ad_count,lambda1,lambda2):
    itera=0
    iteration=k*len(mg_assign)
    while itera<30:
        itera+=1
        gain=-1000
        mid=''
        cid=''
        for cid in c['id']:
            for mid in mg['id']:
                if (mg_count[mid]<k) and (mid not in ad_assign[cid]):
                    gain_dist=float(dxj_avg(cid,mid,dist,c,mg,mg_assign))
                    gain_c_mg=float(lambda1)*c_mg_score(cid,mid,cid_mg_dist)
                    gain_x_x=-float(lambda2)*x_x(cid,mid,ad_count)
                    
                    gain_temp=gain_dist+gain_c_mg+gain_x_x
                    if gain_temp>gain:
                        cid_0=cid
                        mid_0=mid
                        gain=gain_temp
        mg_assign[mid_0].append(cid_0)
        mg_count[mid_0]+=1
        ad_assign[cid_0].append(mid_0)
        ad_count[cid_0]+=1

In [544]:
gain=-1000
mid=''
cid=''
for cid in c['id']:
    for mid in mg['id']:
        if mg_count[mid]<k:
            gain_dist=float(dxj_avg(cid,mid,dist,c,mg,mg_assign))
            gain_c_mg=float(lambda1)*c_mg_score(cid,mid,cid_mg_dist)
            gain_x_x=-float(lambda2)*x_x(cid,mid,ad_count)
                    
            gain_temp=gain_dist+gain_c_mg+gain_x_x
            if gain_temp>gain:
                cid_0=cid
                mid_0=mid
                gain=gain_temp
mg_assign[mid_0].append(cid_0)
mg_count[mid_0]+=1
ad_assign[cid_0].append(mid_0)
ad_count[cid_0]+=1

print gain,cid_0,mid_0

5.9 560046b60b9f73e808486997 56040284767e372b056e0e05


In [564]:
ad_assign

defaultdict(<type 'list'>, {'560046b60b9f73e808486986': [], '560046b60b9f73e808486987': [], '560046b60b9f73e808486985': [], '560046b60b9f73e808486988': [], '560046b60b9f73e808486989': [], '560046b60b9f73e80848699a': [], '560046b60b9f73e80848699b': [], '561d43438c943b5f1225088b': [], '561d43438c943b5f1225088c': [], '560046b60b9f73e808486995': [], '560046b60b9f73e808486994': [], '560046b60b9f73e808486997': ['56040284767e372b056e0e05'], '560046b60b9f73e808486996': [], '560046b60b9f73e808486991': [], '560046b60b9f73e808486990': [], '560046b60b9f73e808486993': [], '560046b60b9f73e808486992': [], '560046b60b9f73e808486999': [], '560046b60b9f73e808486998': [], '560046b60b9f73e80848698b': [], '560046b60b9f73e80848698c': [], '560046b60b9f73e80848698a': [], '560046b60b9f73e80848698f': [], '560046b60b9f73e80848698d': [], '560046b60b9f73e80848698e': []})

In [612]:
flow_assign(c,mg,dist,cid_mg_dist,6,mg_assign,mg_count,ad_assign,ad_count,lambda1/10,lambda2/10)

In [521]:
len(mg_assign)

22

In [613]:
mg_count.values()

[6, 0, 0, 6, 0, 0, 6, 6, 0, 0, 6, 0, 6, 0, 0, 6, 6, 0, 6, 6, 0, 0]

In [614]:
map(lambda x:len(set(x)), ad_assign.values())

[3, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 4, 2, 3, 3, 3, 2, 3, 2, 2]

In [475]:
ad_assign

defaultdict(<type 'list'>, {'560046b60b9f73e808486986': ['560042790b9f73e80848697a', '560042790b9f73e80848697a', '560042790b9f73e80848697c', '5602b65cd1ddf44c37b4bdb8', '5602b501d1ddf44c37b4bdb7'], '560046b60b9f73e808486987': ['560042790b9f73e808486978', '560042790b9f73e808486978', '560042790b9f73e808486978', '560042790b9f73e808486978'], '560046b60b9f73e808486985': ['5604a4c7691b340709b74d35', '560042790b9f73e80848697a', '560042790b9f73e80848697c', '5605bd0bb7be7a9e460c24cf', '5605bd0bb7be7a9e460c24cf'], '560046b60b9f73e808486988': ['560042790b9f73e808486977', '560042790b9f73e80848697a', '5602b65cd1ddf44c37b4bdb8', '5602b65cd1ddf44c37b4bdb8', '5602b501d1ddf44c37b4bdb7'], '560046b60b9f73e808486989': ['560407aa767e372b056e0e0f', '56059082fc5dd7ec2a9a0a53', '56059082fc5dd7ec2a9a0a53', '56059082fc5dd7ec2a9a0a53', '5605bd0bb7be7a9e460c24ce', '5605bd0bb7be7a9e460c24ce', '5605bd0bb7be7a9e460c24ce'], '560046b60b9f73e80848699a': ['56046824691b340709b74d09', '560042790b9f73e80848697c', '5602b65c

In [489]:
c['string'][0]

'Protect Your Home Find Out How'

In [589]:
for k in ad_assign.keys():
    for v in ad_assign[k]:
        ad_idx=c.loc[c['id']==k].index.tolist()
        ad=c['string'].loc[ad_idx[0]]
        mg_idx=mg[mg['id']==v].index.tolist()
        group=mg['display_name'].loc[mg_idx[0]]
        print ad+";"+group

Easy-to-Install Doors Available at The Home Depot;disconnectme
Easy-to-Install Doors Available at The Home Depot;furniture
Easy-to-Install Doors Available at The Home Depot;gym
Easy-to-Install Doors Available at The Home Depot;tool
Easy-to-Install Doors Available at The Home Depot;room
Hassle-Free Installation Try Easy Install Doors;house
Hassle-Free Installation Try Easy Install Doors;weather
Hassle-Free Installation Try Easy Install Doors;gym
Hassle-Free Installation Try Easy Install Doors;recipe
Hassle-Free Installation Try Easy Install Doors;health
Storm-Ready in 45 Minutes Get Prepared at The Home Depot;disconnectme
Storm-Ready in 45 Minutes Get Prepared at The Home Depot;you
Storm-Ready in 45 Minutes Get Prepared at The Home Depot;gym
Storm-Ready in 45 Minutes Get Prepared at The Home Depot;root veggie
Storm-Ready in 45 Minutes Get Prepared at The Home Depot;fall food
New Doors in No Time Click to Browse;V home garden
New Doors in No Time Click to Browse;furniture
New Doors in No

In [420]:
print 1-dist.loc[[1,3,5,7],(1,3,5,7)].sum().sum()
print 4-dist.loc[[1,3,5,7],(1,3,5,7)].sum().sum()/4
print 1-dist.loc[(1,3,7),(1,3,7)].sum().sum()
print 3-dist.loc[(1,3,7),(1,3,7)].sum().sum()/3


-4.1996815249
2.70007961878
-2.54441627951
1.81852790683


In [412]:
df.loc[df['column_name'].isin(some_values)]

1    560046b60b9f73e808486986
3    560046b60b9f73e808486990
5    560046b60b9f73e808486993
7    560046b60b9f73e808486999
Name: id, dtype: object

In [418]:
c.loc[c['id'].isin(['560046b60b9f73e808486986','560046b60b9f73e808486990'])].index.tolist()

[1, 3]

In [265]:
dist[c[c['id']=='560046b60b9f73e80848698c'].index.tolist()[0]][c[c['id']=='560046b60b9f73e80848698c'].index.tolist()[0]]

1.0

In [417]:
c[c['id']=='560046b60b9f73e80848698c'].index

Int64Index([4], dtype='int64')

In [494]:
ad_idx=c.loc[c['id']=='560046b60b9f73e80848698c'].index.tolist()

In [495]:
ad_idx

[4]

In [519]:
len(set(mg['id']))

22

In [520]:
ad_assign['560046b60b9f73e808486988']

['560042790b9f73e808486977',
 '560042790b9f73e80848697a',
 '5602b65cd1ddf44c37b4bdb8',
 '5602b65cd1ddf44c37b4bdb8',
 '5602b501d1ddf44c37b4bdb7']